# RL for node deployment

### Environment
The example is modifed from the Taxi Problem in "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition" by  Tom Dietterich (2000), Journal of Artificial Intelligence Research.

<!--img style="float: right;" src="images/DuckieNav-v1.png"  width="240" height="240"-->


```
MAP = [
    "+-----------------+",
    "| | | : : : : : : |",
    "| | | | | | | | | |",
    "| | : | | | | | | |",
    "| : | | | : : : : |",
    "| | | | | | | | | |",
    "| : : : : : : | | |",
    "| | | | | | | | | |",
    "| | | : : | | : : |",
    "| | | | | | | | | |",
    "| : : : : : : | | |",
    "| | | | | | | | | |",
    "| : : : : : : | | |",
    "| | | | | | | | | |",
    "| : : : : : : : : |",
    "+-----------------+",
]
```

We consider shows a 14 by 9 grid world, except the "service area." The taxi problem is episodic, and in each episode the initial location is located at one of the 4 specially designated locations. 
(1,1), (3,3), (5,5), (7,7). The robot(agent) starts in a given location and must go to some location and deploy 2 nodes with maximum coverage. The episode ends when 2 nodes is deployed.

Adapted from https://www.oreilly.com/learning/introduction-to-reinforcement-learning-and-openai-gym


## 1. Initialize ENV

### Installation
```
$ cd ~/gym-duckienav
$ git pull
$ pip install -e . # you may need sudo depending on your setup
```

In [1]:
import gym
import pickle
import gym_duckienav
import gym.spaces
import numpy as np

env = gym.make("DeployNav-v0")

### How many 'states' in observation_space: 
There are 16003008 states from: (14 (rows) x 9 (columns) ^ 3) x 4 (initial location) x 2 (deploy number)

In [35]:
env.observation_space.n

16003008

### action_space: 
There are 5 possible actions.
* down (0), up (1), right (2), left (3), deploy (4)

In [36]:
env.action_space.n

5

Save env to avoid initialization

In [37]:
#filehandler = open('comm_nav_14_9_16003008.pkl', 'wb') 
#pickle.dump(env, filehandler)
#filehandler.close()

## 2. States

Resets the state of the environment and returns an initial observation (state).

The current state is from :
* initial location
* how many node is dropped
* 
* 
* 

In [42]:
env.reset()

print ("Current state: " + str(env.s))
for p in env.decode(env.s): print (p)
env.render()

Current state: 4160782
1
0
1
1
1
1
1
1
[1, 0]
+-----------------+
|_|_|_:_: : : : : |
|_|_|_|_| | | | | |
|_|_:_|_| | | | | |
|_:_|_|_| : : : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| | | : : | | : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : : : |
+-----------------+



Repeat previous cell for a few times.

In taxi problem, the colors mean:
* blue: passenger's current position
* magenta: destination
* yellow: empty taxi
* green: full taxi

## 3. Actions

Remember that the taxi agent can perform the following actions:
* 0: "South", 
* 1: "North", 
* 2: "East", 
* 3: "West", 
* 4: "Pickup", 
* 5: "Dropoff"

Let's set the state to 124.
Let the taxi agent perform some actions.  

### `step()`

Run one timestep of the environment's dynamics. 
It returns a tuple (observation, reward, done, info)
* observation (object): agent's observation of the current environment
* reward (float) : amount of reward returned after previous action
* done (boolean): whether the episode has ended, in which case further step() calls will return undefined results
* info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)

Essentially the empty taxi is supposed to: 
* move toward the blue letter, 
* pickup the passenger (now the taxi is green), 
* drive to the magenta letter, and 
* drop the passenger (the taxi is yellow again).

It is obvious that we should start with moving "East" env.step(2). Index 2 is for moving "East"
We will do the followings:
* Perform "Pickup" step(4) (although the passenger is not here)
* Perform "East" step(2)
* Perform "Pickup" step(4)
* Perform "West" step(3)
* Perform "South" step(0) for 5 times
* Perfomr "Dropoff" (5)
* Perform "West" step(3) for 4 times
* Perfomr "Dropoff" (5)

In [43]:
state, reward, done, info = env.step(0)
for p in env.decode(env.s): print (p)
env.render()
print ("reward: " + str(reward))

1
0
2
1
1
1
1
1
[1, 0]
+-----------------+
|_|_|_:_: : : : : |
|_|_|_|_| | | | | |
|_|_:_|_| | | | | |
|_:_|_|_| : : : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| | | : : | | : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : : : |
+-----------------+
  (South)
reward: -1


In [44]:
state, reward, done, info = env.step(4)
for p in env.decode(env.s): print (p)
env.render()
print ("reward: " + str(reward))
print ("done: " + str(done))

1
1
2
1
2
1
1
1
[1, 0]
+-----------------+
|_|_|_:_: : : : : |
|_|_|_|_| | | | | |
|_|_:_|_| | | | | |
|_:_|_|_| : : : : |
|_|_|_|_| | | | | |
| : : : : : : | | |
| | | | | | | | | |
| | | : : | | : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : : : |
+-----------------+
  (Deploy)
reward: 40
done: False


In [45]:
state, reward, done, info = env.step(2)
for p in env.decode(env.s): print (p)
env.render()
print ("reward: " + str(reward))

1
1
2
2
2
1
1
1
[1, 0]
+-----------------+
|_|_|_:_: : : : : |
|_|_|_|_| | | | | |
|_|_:_|_| | | | | |
|_:_|_|_| : : : : |
|_|_|_|_| | | | | |
| : : : : : : | | |
| | | | | | | | | |
| | | : : | | : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : : : |
+-----------------+
  (East)
reward: -1


In [46]:
state, reward, done, info = env.step(4)
for p in env.decode(env.s): print (p)
env.render()
print ("reward: " + str(reward))
print ("done: " + str(done))

1
1
2
2
2
1
2
2
[1, 0]
+-----------------+
|_|_|_:_:_: : : : |
|_|_|_|_|_| | | | |
|_|_:_|_|_| | | | |
|_:_|_|_|_: : : : |
|_|_|_|_|_| | | | |
| : : : : : : | | |
| | | | | | | | | |
| | | : : | | : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : : : |
+-----------------+
  (Deploy)
reward: 50
done: True


### Rewards

You have probably figured out the rewards:
* Perform any movements: -1
* Pick up or drop off at the wrong position: -10
* Drop off the passenger at the right position: 20 

## 4. Random Agent: 

We will use the funciton env.action_space.sample(); you could run the following cell for a few times

In [47]:
print (env.action_space.sample())

4


### How good does behaving completely random do?

In [48]:
state = env.reset()

done = None
counter = 0
g = 0
reward = None
while done != True:
    state, reward, done, info = env.step(env.action_space.sample())
    counter += 1
    g += reward
print("Solved in {} Steps with a total reward of {}".format(counter,g))


Solved in 5 Steps with a total reward of -203


You may luck out and solve the environment fairly quickly, but on average, a completely random policy will solve this environment in about ???? steps

## 5. Agent with Basic Reinforcement Learning: Q-Learning

In order to maximize our reward, we will have to have the algorithm remember its actions and their associated rewards. Here, the algorithm’s memory is going to be a Q action value table.

To manage this Q table, we will use a NumPy array. The size of this table will be the number of states (2520) by the number of possible actions (6).

In [49]:
n_states = env.observation_space.n
n_actions = env.action_space.n
Q = np.zeros([n_states, n_actions])

episodes = 1
G = 0
counter = 0
alpha = 0.05

for episode in range(1,episodes+1):
    done = False
    G, reward = 0,0

    state = env.reset()
    env.render()
    
    firstState = state
    print("Initial State = {}".format(state))
    while done != True:
        action = np.argmax(Q[state])  #1
        state2, reward, done, info = env.step(action) #2
        Q[state,action] += alpha * (reward + np.max(Q[state2]) - Q[state,action]) #3
        
        if counter < 100:
            print("Step: {}, Action: {}, Reward: {}, Q[{}] \t{}".format(counter, action, reward, state, Q[state]))

        counter += 1
        G += reward
        state = state2
        
finalState = state
print("Final State = {}".format(finalState))
print("Solved in {} Steps with a total reward of {}".format(counter, G))

print (Q[finalState])

[0, 0]
+-----------------+
|_|_|_: : : : : : |
|_|_|_| | | | | | |
|_|_:_| | | | | | |
| : | | | : : : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| | | : : | | : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : : : |
+-----------------+

Initial State = 0
Step: 0, Action: 0, Reward: -1, Q[0] 	[-0.05  0.    0.    0.    0.  ]
Step: 1, Action: 0, Reward: -1, Q[142884] 	[-0.05  0.    0.    0.    0.  ]
Step: 2, Action: 0, Reward: -50, Q[285768] 	[-2.5  0.   0.   0.   0. ]
Step: 3, Action: 0, Reward: -50, Q[428652] 	[-2.5  0.   0.   0.   0. ]
Step: 4, Action: 0, Reward: -50, Q[571536] 	[-2.5  0.   0.   0.   0. ]
Step: 5, Action: 0, Reward: -50, Q[714420] 	[-2.5  0.   0.   0.   0. ]
Step: 6, Action: 0, Reward: -50, Q[857304] 	[-2.5  0.   0.   0.   0. ]
Step: 7, Action: 0, Reward: -50, Q[1000188] 	[-2.5  0.   0.   0.   0. ]
Step: 8, Action: 0, Reward: -50, Q[1143072] 	[-2.5  0.   0.   0.   0. ]
Step: 9, A

First (#1): The agent starts by choosing an action with the highest Q value for the current state using argmax. Argmax will return the index/action with the highest value for that state. Initially, our Q table will be all zeros. But, after every step, the Q values for state-action pairs will be updated.

Second (#2): The agent then takes action and we store the future state as state2 (S t+1). This will allow the agent to compare the previous state to the new state.

Third (#3): We update the state-action pair (St , At) for Q using the reward, and the max Q value for state2 (S t+1). This update is done using the action value formula (based upon the Bellman equation) and allows state-action pairs to be updated in a recursive fashion (based on future values). See the following Figure for the value iteration update.

<img src="images/qlearn.png">

### Let's run over multiple episodes so that we can converge on a optimal policy

In [50]:
episodes = 500
rewardTracker = []

G = 0
alpha = 0.05

for episode in range(1,episodes+1):
    done = False
    G, reward = 0,0

    state = env.reset()

    while done != True:
        action = np.argmax(Q[state]) 
        state2, reward, done, info = env.step(action) 
        Q[state,action] += alpha * ((reward + (np.max(Q[state2]))  - Q[state,action]))
        G += reward
        state = state2
    
    if episode % 5 == 0:
        print('Episode {} Total Reward: {}'.format(episode,G))
    


Episode 5 Total Reward: -3834
Episode 10 Total Reward: -862
Episode 15 Total Reward: 58
Episode 20 Total Reward: 58
Episode 25 Total Reward: -2021
Episode 30 Total Reward: -866
Episode 35 Total Reward: 58
Episode 40 Total Reward: 98
Episode 45 Total Reward: 98
Episode 50 Total Reward: 98
Episode 55 Total Reward: 78
Episode 60 Total Reward: 78
Episode 65 Total Reward: 98
Episode 70 Total Reward: 78
Episode 75 Total Reward: 98
Episode 80 Total Reward: 58
Episode 85 Total Reward: 98
Episode 90 Total Reward: 98
Episode 95 Total Reward: 58
Episode 100 Total Reward: 98
Episode 105 Total Reward: 58
Episode 110 Total Reward: 98
Episode 115 Total Reward: 98
Episode 120 Total Reward: 58
Episode 125 Total Reward: 98
Episode 130 Total Reward: 98
Episode 135 Total Reward: 98
Episode 140 Total Reward: 98
Episode 145 Total Reward: 58
Episode 150 Total Reward: 58
Episode 155 Total Reward: 58
Episode 160 Total Reward: 98
Episode 165 Total Reward: 78
Episode 170 Total Reward: 58
Episode 175 Total Reward

### Now that we have learned the optimal Q Values we have developed a optimal policy and have no need to train the agent anymore

In [51]:
state = env.reset()

done = None

while done != True:
    # We simply take the action with the highest Q Value
    action = np.argmax(Q[state])
    state, reward, done, info = env.step(action)
    env.render()

[2, 0]
+-----------------+
|_|_|_:_:_: : : : |
|_|_|_|_|_| | | | |
|_|_:_|_|_| | | | |
|_:_|_|_|_: : : : |
|_|_|_|_|_| | | | |
| : : : : : : | | |
| | | | | | | | | |
| | | : : | | : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : : : |
+-----------------+
  (South)
[2, 0]
+-----------------+
|_|_|_:_:_: : : : |
|_|_|_|_|_| | | | |
|_|_:_|_|_| | | | |
|_:_|_|_|_: : : : |
|_|_|_|_|_| | | | |
|_:_:_:_:_: : | | |
| | | | | | | | | |
| | | : : | | : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : : : |
+-----------------+
  (Deploy)
[2, 0]
+-----------------+
|_|_|_:_:_: : : : |
|_|_|_|_|_| | | | |
|_|_:_|_|_| | | | |
|_:_|_|_|_: : : : |
|_|_|_|_|_| | | | |
|_:_:_:_:_: : | | |
| | | | | | | | | |
| | | : : | | : : |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : | | |
| | | | | | | | | |
| : : : : : : : : |
+-----------------